In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [2]:
from langchain.agents import tool

@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor

# 도구 정의
tools = [add_function]

# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# prompt 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Make sure to use the `search_news` tool for searching ketword related news."),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Agent 생성
llm_agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=llm_agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True,
)

In [4]:
# 계산할 질문 설정
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

# agent_executor를 반복적으로 실행
for step in agent_executor.iter({"input": question}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "add_function":
            # Tool 실행 결과 출력
            print(f"\nTool Name: {action.tool}, 실행 결과: {value}")
        # 사용자에게 계속 진항할지 물음
        _continue = input("계속 진행하시겠습니까? (y/n)?:\n") or "Y"
        # 사용자가 'y'가 아닌 다른 입력을 하면 반복 중단
        if _continue.lower() != 'y':
            break

# 최종 결과 출력
if "output" in step:
    print(step["output"])



> Entering new None chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Tool Name: add_function, 실행 결과: 235.7

Invoking: `add_function` with `{'a': 235.7, 'b': 34.2}`


269.9
Tool Name: add_function, 실행 결과: 269.9

Invoking: `add_function` with `{'a': 269.9, 'b': 110.1}`


380.0
Tool Name: add_function, 실행 결과: 380.0
계산 결과는 380.0입니다.

> Finished chain.
계산 결과는 380.0입니다.
